# Criação das Tabelas Trust

### Configurações Iniciais

In [2]:
from pyspark.sql.types import *
import pyspark.sql.functions as fn
from pyspark.sql import SparkSession
from IPython.core.display import HTML

display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Define a sessão do Spark com os jars necessários para conexão com o MINIO
spark = (SparkSession.builder
         .config("spark.jars","""/home/jovyan/jars/aws-java-sdk-core-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-dynamodb-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-s3-1.11.534.jar,
                                 /home/jovyan/jars/hadoop-aws-3.2.2.jar""")
         .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0")
         .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
         .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
         .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
         .config("spark.hadoop.fs.s3a.access.key", "aulafia")
         .config("spark.hadoop.fs.s3a.secret.key", "aulafia@123")
         .config("spark.hadoop.fs.s3a.path.style.access", True)
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
         .getOrCreate()
        )

from delta.tables import DeltaTable

## Games by Platform

### Leitura das bases

In [61]:
path_games = 's3a://context/igdb/games/delta/'
path_platforms = 's3a://context/igdb/platforms/delta/'

dfIGDB_Games = DeltaTable.forPath(spark, path_games).toDF()
dfIGDB_Platforms = DeltaTable.forPath(spark, path_platforms).toDF()

### Explode das Plataformas

In [69]:
# Exibe o DataFrame resultante
dfExploded = (dfIGDB_Games
 .select(
     fn.col("id").alias("Game_id"),
     fn.col("name").alias("Game_Name"),
     fn.explode_outer("platforms").alias("Platform_id"))
)

dfExploded.show(truncate=False)

+-------+-------------------------------------+-----------+
|Game_id|Game_Name                            |Platform_id|
+-------+-------------------------------------+-----------+
|1      |Thief II: The Metal Age              |6          |
|5      |Baldur's Gate                        |3          |
|5      |Baldur's Gate                        |6          |
|5      |Baldur's Gate                        |14         |
|5      |Baldur's Gate                        |39         |
|6      |Baldur's Gate II: Shadows of Amn     |3          |
|6      |Baldur's Gate II: Shadows of Amn     |6          |
|6      |Baldur's Gate II: Shadows of Amn     |14         |
|7      |Jagged Alliance                      |13         |
|7      |Jagged Alliance                      |20         |
|9      |Jagged Alliance 2                    |3          |
|9      |Jagged Alliance 2                    |6          |
|9      |Jagged Alliance 2                    |16         |
|17     |Fallout Tactics: Brotherhood of

In [70]:
dfExploded.select("game_id").distinct().count()

257084

### Tabela Final (Trust)

In [71]:
path_gbp = 's3a://trust/games_by_platform/delta/'

df_GBP = DeltaTable.forPath(spark, path_gbp).toDF()
df_GBP.show()

+-------+--------------------+-----------------+---------------+-------------------+-----------+--------------------+-------------------+------------+
|game_id|           game_name|platform_category|platform_family|platform_generation|platform_id|       platform_name|       release_date|release_year|
+-------+--------------------+-----------------+---------------+-------------------+-----------+--------------------+-------------------+------------+
|      1|Thief II: The Met...| operating_system| (Não Definida)|     (Não Definida)|          6|PC (Microsoft Win...|2000-03-21 00:00:00|        2000|
|      5|       Baldur's Gate| operating_system|          Linux|     (Não Definida)|          3|               Linux|2015-10-14 00:00:00|        2015|
|      5|       Baldur's Gate| operating_system| (Não Definida)|     (Não Definida)|          6|PC (Microsoft Win...|1998-12-21 00:00:00|        1998|
|      5|       Baldur's Gate| operating_system| (Não Definida)|     (Não Definida)|         1

In [72]:
total_games = df_GBP.select(fn.count("game_id").alias("total_games")).collect()[0]["total_games"]
print(f"O número total de jogos é: {total_games}")

O número total de jogos é: 324364


In [73]:
# Contar o número de game_id distintos na tabela df_games_by_platform
distinct_game_ids = df_GBP.select(fn.countDistinct("game_id").alias("distinct_game_ids")).collect()[0]["distinct_game_ids"]

# Mostrar o número de game_id distintos
print(f"O número de game_id distintos é: {distinct_game_ids}")

O número de game_id distintos é: 220943


In [67]:
df_GBP.select("game_id").distinct().count()

173019

In [65]:
dfIGDB_Games.select("id").distinct().count()

257084

In [7]:
path_releaseDates = 's3a://context/igdb/release_dates/delta/'

dfIGDB_rDates = DeltaTable.forPath(spark, path_releaseDates).toDF()
dfIGDB_rDates.show()

+--------+-------------------+-------------------+------+------------+------+----+-----------+--------------------+------+------------+-------------------+----+
|category|         created_at|               date|  game|       human|    id|   m|platform_id|       platform_name|region|      status|         updated_at|   y|
+--------+-------------------+-------------------+------+------------+------+----+-----------+--------------------+------+------------+-------------------+----+
|       0|2023-09-18 19:40:41|2013-06-11 00:00:00| 52220|Jun 11, 2013|517124|   6|         46|    PlayStation Vita|     2|        null|2023-09-18 20:45:33|2013|
|       0|2023-09-18 19:51:23|2002-08-29 00:00:00|    39|Aug 29, 2002|517125|   8|          6|PC (Microsoft Win...|     2|        null|2023-09-18 20:31:03|2002|
|       0|2023-09-18 19:51:23|2004-02-04 00:00:00|    39|Feb 04, 2004|517127|   2|          8|       PlayStation 2|     2|        null|2023-09-18 20:31:03|2004|
|       0|2023-09-18 19:51:23|2004

### Enriquecendo com dados de lançamento

In [21]:
from pyspark.sql.window import Window

# Identificar o menor ano de lançamento para cada combinação de game_id e platform_id
window_spec = Window().partitionBy("game", "platform_id").orderBy(fn.col("y"))
min_year_df = dfIGDB_rDates.withColumn("min_release_year", fn.min("y").over(window_spec))

min_year_df.show()

+--------+-------------------+-------------------+----+------------+------+---+-----------+--------------------+------+------+-------------------+----+----------------+
|category|         created_at|               date|game|       human|    id|  m|platform_id|       platform_name|region|status|         updated_at|   y|min_release_year|
+--------+-------------------+-------------------+----+------------+------+---+-----------+--------------------+------+------+-------------------+----+----------------+
|       0|2015-01-11 11:24:23|1998-11-30 00:00:00|   2|Nov 30, 1998| 25214| 11|          6|PC (Microsoft Win...|     2|  null|2015-01-11 11:49:24|1998|            1998|
|       2|2015-01-11 11:24:23|1998-12-31 00:00:00|   2|        1998| 25215| 12|          6|PC (Microsoft Win...|     1|  null|2015-01-11 11:49:24|1998|            1998|
|       0|2018-01-19 16:43:51|2014-02-25 00:00:00|   4|Feb 25, 2014|136391|  2|         49|            Xbox One|     2|  null|2018-01-20 17:42:48|2014|    

In [38]:
# Juntar os DataFrames para enriquecer a tabela original
df = (
    df_GBP
    .join(
        min_year_df,
        (df_GBP.game_id == min_year_df.game)
        & (df_GBP.platform_id == min_year_df.platform_id),
        "left_outer"
    )
    .select("game_id", "game_name", df_GBP.platform_id, df_GBP.platform_name, fn.col("min_release_year").alias("release_year")) 
    .orderBy("game_id")
)

df = (
    df
    .withColumn("row_number", fn.row_number().over(Window.partitionBy("game_id", "platform_id").orderBy("release_year")))
    .filter(fn.col("row_number") == 1)
    .drop("row_number")
    .orderBy("game_id")
)

df.show()

+-------+--------------------+-----------+--------------------+------------+
|game_id|           game_name|platform_id|       platform_name|release_year|
+-------+--------------------+-----------+--------------------+------------+
|      1|Thief II: The Met...|          6|PC (Microsoft Win...|        2000|
|      2|Thief: The Dark P...|          6|PC (Microsoft Win...|        1998|
|      3|Thief: Deadly Sha...|          6|PC (Microsoft Win...|        2004|
|      3|Thief: Deadly Sha...|         11|                Xbox|        2004|
|      4|               Thief|         49|            Xbox One|        2014|
|      4|               Thief|          6|PC (Microsoft Win...|        2014|
|      4|               Thief|         12|            Xbox 360|        2014|
|      4|               Thief|          9|       PlayStation 3|        2014|
|      4|               Thief|         48|       PlayStation 4|        2014|
|      4|               Thief|         14|                 Mac|        2014|

In [48]:
path_releaseDates = 's3a://context/igdb/release_dates/delta/'

df_release_dates = DeltaTable.forPath(spark, path_releaseDates).toDF()


df_release_years = (
        df_release_dates
        .select(
            fn.col("game").alias("game_id"),
            fn.col("platform_id"),
            fn.col('region'),
            fn.col('date'),
            fn.col('y')
        )
        .groupBy("game_id", "platform_id")
        .agg(
            fn.min(df_release_dates.y).alias('release_year'),
            fn.min(df_release_dates.date).alias('release_date'),
        )
        .orderBy("game_id")
    )

df_release_years.show()

+-------+-----------+------------+-------------------+
|game_id|platform_id|release_year|       release_date|
+-------+-----------+------------+-------------------+
|      1|          6|        2000|2000-03-21 00:00:00|
|      2|          6|        1998|1998-11-30 00:00:00|
|      3|          6|        2004|2004-05-25 00:00:00|
|      3|         11|        2004|2004-05-25 00:00:00|
|      4|         49|        2014|2014-02-25 00:00:00|
|      4|         48|        2014|2014-02-25 00:00:00|
|      4|          9|        2014|2014-02-25 00:00:00|
|      4|          6|        2014|2014-02-27 00:00:00|
|      4|         14|        2014|2014-02-27 00:00:00|
|      4|         12|        2014|2014-02-25 00:00:00|
|      5|          6|        1998|1998-12-21 00:00:00|
|      5|          3|        2015|2015-10-14 00:00:00|
|      5|         14|        2015|2015-10-14 00:00:00|
|      5|         39|        2012|2012-11-30 00:00:00|
|      6|          6|        2000|2000-09-21 00:00:00|
|      6| 